# HW#6 Regularization

안녕하세요, 광운대학교 로봇학부의 오정현 교수입니다. 본 자료는 딥러닝 실습 수업을 위해 제작된 것입니다.

파이썬 문법
- 점프투파이썬(https://wikidocs.net/book/1) 참고

이번 과제는 딥러닝의 일반화 성능을 높이기 위한 Regularization을 해보는 것입니다.이미지 분류에 여러 가지 Regularization 기법을 적용해 보도록 하겠습니다. 대표적인 Regularization 기법으로 Dropout, Data augmentation, Batch Normalization 등이 있습니다.

이번 과제는 (https://www.tensorflow.org/tutorials/keras/classification?hl=ko)를 참고하면 좋습니다.

#1. Data Generation
Data는 mnist dataset을 이용하도록 하겠습니다. mnist dataset은 원래 60000개의 training set이 주어져 있지만 overfitting을 유도하기 위하여 1000개의 data만 이용하려고 합니다. 1000개의 data로 이루어진 x_train과 y_train을 만들어 보세요. 그리고 2000개로 이루어진 large_x_train, large_y_train을 만들어보세요. 그리고 training data의 다른 범위에서 200개로 이루어진 x_validation과 y_validation도 만들어 보세요.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Flatten
from keras import backend as K
from matplotlib import pyplot

batch_size = 28
num_classes = 10
epochs = 100

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(X_train, Y_train), (x_test,y_test) = mnist.load_data()

### START CODE HERE ###
x_validation = X_train[1500:1700]
y_validation = Y_train[1500:1700]
large_x_train = X_train[2000:4000]
large_y_train = Y_train[2000:4000]
x_train = X_train[:1000]
y_train = Y_train[:1000]
### END CODE HERE ###
print("x_validation shape:", x_validation.shape)
print("y_validation shape:", y_validation.shape)
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    large_x_train = large_x_train.reshape(large_x_train.shape[0], 1, img_rows, img_cols)
    x_validation = x_validation.reshape(x_validation.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    large_x_train = large_x_train.reshape(large_x_train.shape[0], img_rows, img_cols, 1)
    x_validation = x_validation.reshape(x_validation.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
large_x_train = large_x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
large_x_train /= 255
x_validation /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
large_y_train = keras.utils.to_categorical(large_y_train, num_classes)
y_validation = keras.utils.to_categorical(y_validation, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

assert large_x_train.shape[0]==2000
assert large_y_train.shape[0]==2000
assert x_train.shape[0]==1000
assert y_train.shape[0]==1000
assert x_validation.shape[0]==200
assert y_validation.shape[0]==200

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("large_x_train shape:", large_x_train.shape)
print("large_y_train shape:", large_y_train.shape)
print("x_validation shape:", x_validation.shape)
print("y_validation shape:", y_validation.shape)

11493376/11490434 [==============================] - 0s 0us/step
x_validation shape: (200, 28, 28)
y_validation shape: (200,)
x_train shape: (1000, 28, 28, 1)
y_train shape: (1000, 10)
large_x_train shape: (2000, 28, 28, 1)
large_y_train shape: (2000, 10)
x_validation shape: (200, 28, 28, 1)
y_validation shape: (200, 10)


#2. 모델 생성
복습 차원에서 MLP 분류모델을 만들어 보도록 하겠습니다. 모델의 마지막 레이어에는 활성화 함수로 10개의 출력을 가지는 softmax를 달겠습니다. 이를 통해서 모델은 이미지안의 숫자가 0부터 9까지의 숫자중에 어디에 가까운지를 확률적으로 나타냅니다. 



다음과 같은 MLP 모델을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| Dense | (None, 1024) | 1049600 |
| Dense | (None, 1024) | 1049600 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [2]:
model = Sequential()
### START CODE HERE ###
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))
### END CODE HERE ###

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              803840    
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
Total params: 2,913,290
Trainable params: 2,913,290
Non-trainable params: 0
______________________________________________

#3. Learning MLP
기본 MLP 분류모델을 학습해 보겠습니다. Overfitting은 Training data에 맞추어 과도하게 학습이 이루어져 Test data에서 높은 성능이 나지 않는 현상, 즉 Generalization 성능이 높지 않게 나타나는 현상을 의미합니다. 따라서 Overfitting이 발생하면 Training accuracy는 높지만 Test accuracy는 높지 않게 나타납니다. 

현재 모델이 overfitting이 발생하는지 체크해 보세요. 일부러 overfitting이 발생하도록 유도하였기 때문에 overfitting 현상이 나타나야 합니다.

In [3]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
weights = model.get_weights()

history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
36/36 [==============================] - 2s 46ms/step - loss: 2.3155 - accuracy: 0.0746 - val_loss: 2.2973 - val_accuracy: 0.1050
Epoch 2/100
36/36 [==============================] - 1s 32ms/step - loss: 2.2993 - accuracy: 0.0854 - val_loss: 2.2876 - val_accuracy: 0.1300
Epoch 3/100
36/36 [==============================] - 1s 33ms/step - loss: 2.2962 - accuracy: 0.0991 - val_loss: 2.2780 - val_accuracy: 0.1400
Epoch 4/100
36/36 [==============================] - 1s 33ms/step - loss: 2.2746 - accuracy: 0.1449 - val_loss: 2.2685 - val_accuracy: 0.1750
Epoch 5/100
36/36 [==============================] - 1s 33ms/step - loss: 2.2664 - accuracy: 0.1638 - val_loss: 2.2590 - val_accuracy: 0.2150
Epoch 6/100
36/36 [==============================] - 1s 33ms/step - loss: 2.2477 - accuracy: 0.2126 - val_loss: 2.2496 - val_accuracy: 0.2450
Epoch 7/100
36/36 [==============================] - 1s 33ms/step - loss: 2.2315 - accuracy: 0.2711 - val_loss: 2.2404 - val_accuracy: 0.2600
Epoch 

#4. Regularization
Overfitting이 발생한 모델에 다양한 Regularization 기법을 이용해 보도록 합시다.

4.1 Large Dataset

Training data가 충분하다면 overfitting 현상이 발생할 가능성이 줄어듭니다. 기본 MLP 분류 모델에서 large_x_train과 large_y_train을 이용하면 성능이 올라갈 것입니다. Generalization 성능이 올라갔나요?

In [4]:
#initialize weights
model.set_weights(weights)

large_model_history=model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = model.evaluate(x_test, y_test, verbose=0)
print('Largemodel Test loss:', score[0])
print('Largemodel Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 2s 33ms/step - loss: 2.3016 - accuracy: 0.0905 - val_loss: 2.2875 - val_accuracy: 0.1300
Epoch 2/100
72/72 [==============================] - 2s 32ms/step - loss: 2.2790 - accuracy: 0.1315 - val_loss: 2.2686 - val_accuracy: 0.1650
Epoch 3/100
72/72 [==============================] - 2s 31ms/step - loss: 2.2573 - accuracy: 0.1730 - val_loss: 2.2501 - val_accuracy: 0.2150
Epoch 4/100
72/72 [==============================] - 2s 32ms/step - loss: 2.2360 - accuracy: 0.2215 - val_loss: 2.2319 - val_accuracy: 0.2750
Epoch 5/100
72/72 [==============================] - 2s 32ms/step - loss: 2.2151 - accuracy: 0.2665 - val_loss: 2.2139 - val_accuracy: 0.3150
Epoch 6/100
72/72 [==============================] - 2s 31ms/step - loss: 2.1946 - accuracy: 0.3165 - val_loss: 2.1961 - val_accuracy: 0.3450
Epoch 7/100
72/72 [==============================] - 2s 32ms/step - loss: 2.1741 - accuracy: 0.3780 - val_loss: 2.1784 - val_accuracy: 0.3750
Epoch 

##4.2 Dropout
Dropout은 쉽게 쓸 수 있는 Regularization 기법입니다. Layer 사이에 Dropout layer만 추가하면 되기 때문에 간편합니다. 다음과 같은 Dropout model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| Dense | (None, 1024) | 1049600 |
| Dense | (None, 1024) | 1049600 |
| Dropout | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [5]:
dropout_model = Sequential()
### START CODE HERE ###
dropout_model.add(keras.layers.Flatten(input_shape=(28, 28)))
dropout_model.add(keras.layers.Dense(1024,activation = 'relu'))
dropout_model.add(keras.layers.Dense(1024,activation = 'relu'))
dropout_model.add(keras.layers.Dense(1024,activation = 'relu'))
dropout_model.add(keras.layers.Dropout(0.2))
dropout_model.add(keras.layers.Flatten())
dropout_model.add(keras.layers.Dense(10, activation='softmax'))
### END CODE HERE ###
dropout_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

Dropout Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

※Dropout을 적용한 Model은 일정확률로 신경망의 뉴런을 비활성화시키기 때문에, 오버피팅을 방지하는 효과가 있습니다. 하지만 비활성화로 인해서  학습속도가 떨어진다는 단점이 존재합니다. 그 때문에 test accuracy는 같은 학습파라미터 조건에서 기존모델보다 더 낮을 수 있습니다. 

기존모델과 dropout을 적용한 모델에 대해서, train 데이터와 test데이터에 대한 accuracy차이를 주목해보시면 좋을 것같습니다!

In [6]:
dropout_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

dropout_model_history=dropout_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = dropout_model.evaluate(x_test, y_test, verbose=0)
print('Dropout model Test loss:', score[0])
print('Dropout model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 3s 37ms/step - loss: 2.2998 - accuracy: 0.1281 - val_loss: 2.2673 - val_accuracy: 0.1750
Epoch 2/100
72/72 [==============================] - 2s 34ms/step - loss: 2.2767 - accuracy: 0.1720 - val_loss: 2.2511 - val_accuracy: 0.2350
Epoch 3/100
72/72 [==============================] - 2s 32ms/step - loss: 2.2621 - accuracy: 0.1925 - val_loss: 2.2352 - val_accuracy: 0.2900
Epoch 4/100
72/72 [==============================] - 2s 32ms/step - loss: 2.2329 - accuracy: 0.2622 - val_loss: 2.2196 - val_accuracy: 0.3300
Epoch 5/100
72/72 [==============================] - 2s 33ms/step - loss: 2.2131 - accuracy: 0.3028 - val_loss: 2.2040 - val_accuracy: 0.3700
Epoch 6/100
72/72 [==============================] - 2s 33ms/step - loss: 2.2019 - accuracy: 0.3288 - val_loss: 2.1885 - val_accuracy: 0.3850
Epoch 7/100
72/72 [==============================] - 2s 33ms/step - loss: 2.1820 - accuracy: 0.3666 - val_loss: 2.1729 - val_accuracy: 0.4350
Epoch 

##4.3 BatchNormalization
BatchNormalization(BN)은 쉽게 쓸 수 있는 Regularization 기법입니다. Layer 사이에 BN layer만 추가하면 되기 때문에 간편합니다. 다음과 같은 BN model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [7]:
bn_model = Sequential()
### START CODE HERE ###
bn_model.add(keras.layers.Flatten(input_shape=(28, 28)))
bn_model.add(keras.layers.Dense(1024))
bn_model.add(keras.layers.BatchNormalization())
bn_model.add(keras.layers.Activation('relu'))
bn_model.add(keras.layers.Dense(1024))
bn_model.add(keras.layers.BatchNormalization())
bn_model.add(keras.layers.Activation('relu'))
bn_model.add(keras.layers.Dense(1024))
bn_model.add(keras.layers.BatchNormalization())
bn_model.add(keras.layers.Activation('relu'))
bn_model.add(keras.layers.Flatten())
bn_model.add(keras.layers.Dense(10, activation='softmax'))
### END CODE HERE ###

bn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              803840    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_1 (Activation)    (None, 1024)             

BN Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [8]:
bn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
bn_model_history=bn_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = bn_model.evaluate(x_test, y_test, verbose=0)
print('BN model Test loss:', score[0])
print('BN model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 4s 37ms/step - loss: 2.6297 - accuracy: 0.1141 - val_loss: 2.2875 - val_accuracy: 0.1200
Epoch 2/100
72/72 [==============================] - 2s 33ms/step - loss: 2.3977 - accuracy: 0.1673 - val_loss: 2.2274 - val_accuracy: 0.1600
Epoch 3/100
72/72 [==============================] - 2s 33ms/step - loss: 2.2566 - accuracy: 0.2259 - val_loss: 2.1223 - val_accuracy: 0.2750
Epoch 4/100
72/72 [==============================] - 2s 33ms/step - loss: 2.0935 - accuracy: 0.2735 - val_loss: 1.9851 - val_accuracy: 0.3600
Epoch 5/100
72/72 [==============================] - 2s 33ms/step - loss: 1.9286 - accuracy: 0.3389 - val_loss: 1.8457 - val_accuracy: 0.4500
Epoch 6/100
72/72 [==============================] - 2s 34ms/step - loss: 1.8435 - accuracy: 0.3949 - val_loss: 1.7205 - val_accuracy: 0.5000
Epoch 7/100
72/72 [==============================] - 2s 34ms/step - loss: 1.6971 - accuracy: 0.4570 - val_loss: 1.6105 - val_accuracy: 0.5200
Epoch 

##4.4 Final Model
지금까지 썼던 Regularization 기법들을 종합선물세트로 적용해 봅시다. 다음과 같은 Final model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dropout | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [9]:
final_model = Sequential()
### START CODE HERE ###
final_model.add(keras.layers.Flatten(input_shape=(28, 28)))
final_model.add(keras.layers.Dense(1024))
final_model.add(keras.layers.BatchNormalization())
final_model.add(keras.layers.Activation('relu'))
final_model.add(keras.layers.Dense(1024))
final_model.add(keras.layers.BatchNormalization())
final_model.add(keras.layers.Activation('relu'))
final_model.add(keras.layers.Dense(1024))
final_model.add(keras.layers.BatchNormalization())
final_model.add(keras.layers.Activation('relu'))
final_model.add(keras.layers.Dropout(0.2))
final_model.add(keras.layers.Flatten())
final_model.add(keras.layers.Dense(10, activation='softmax'))
### END CODE HERE ###

final_model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              803840    
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_3 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_4 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_4 (Activation)    (None, 1024)             

Final Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [10]:
final_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
final_model_history=final_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = final_model.evaluate(x_test, y_test, verbose=0)
print('Final model Test loss:', score[0])
print('Final model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 4s 36ms/step - loss: 2.8959 - accuracy: 0.0910 - val_loss: 2.3520 - val_accuracy: 0.1100
Epoch 2/100
72/72 [==============================] - 2s 33ms/step - loss: 2.7803 - accuracy: 0.1059 - val_loss: 2.3506 - val_accuracy: 0.1300
Epoch 3/100
72/72 [==============================] - 2s 33ms/step - loss: 2.5173 - accuracy: 0.1569 - val_loss: 2.2841 - val_accuracy: 0.1700
Epoch 4/100
72/72 [==============================] - 2s 33ms/step - loss: 2.3185 - accuracy: 0.2068 - val_loss: 2.1733 - val_accuracy: 0.2050
Epoch 5/100
72/72 [==============================] - 2s 33ms/step - loss: 2.1803 - accuracy: 0.2470 - val_loss: 2.0457 - val_accuracy: 0.2700
Epoch 6/100
72/72 [==============================] - 2s 33ms/step - loss: 2.0661 - accuracy: 0.2745 - val_loss: 1.9206 - val_accuracy: 0.3350
Epoch 7/100
72/72 [==============================] - 2s 33ms/step - loss: 1.8963 - accuracy: 0.3421 - val_loss: 1.8035 - val_accuracy: 0.4200
Epoch 

(optional) Training data가 늘어나면 regularization 효과가 나는 것을 보였습니다. 쉽게 training data를 늘릴 수 있는 방법은 data augmentation 입니다. 이 방법은 기존 training data를 적절히 rotating, flipping, scaling, shifting 하여 training data 수를 늘리는 것입니다. data augmentation의 regularization 효과를 테스트해 보세요. 또한 뉴럴 네트워크의 노드 개수나 층수를 바꿔서 성능을 올려보는 것도 테스트해보세요.

In [11]:
final_model2 = Sequential()
### START CODE HERE ###
final_model2.add(keras.layers.Flatten(input_shape=(28, 28)))
final_model2.add(keras.layers.Dense(1024))
final_model2.add(keras.layers.BatchNormalization())
final_model2.add(keras.layers.Activation('relu'))
final_model2.add(keras.layers.Dense(1024))
final_model2.add(keras.layers.BatchNormalization())
final_model2.add(keras.layers.Activation('relu'))
final_model2.add(keras.layers.Dense(1024))
final_model2.add(keras.layers.BatchNormalization())
final_model2.add(keras.layers.Activation('relu'))
final_model2.add(keras.layers.Dense(1024))
final_model2.add(keras.layers.BatchNormalization())
final_model2.add(keras.layers.Activation('relu'))
final_model2.add(keras.layers.Dense(1024))
final_model2.add(keras.layers.BatchNormalization())
final_model2.add(keras.layers.Activation('relu'))
final_model2.add(keras.layers.Dropout(0.2))
final_model2.add(keras.layers.Flatten())
final_model2.add(keras.layers.Dense(10, activation='softmax'))

final_model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              803840    
_________________________________________________________________
batch_normalization_6 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_6 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_7 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_7 (Activation)    (None, 1024)             

In [12]:
final_model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
final_model_history2=final_model2.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = final_model2.evaluate(x_test, y_test, verbose=0)
print('Final model Test loss:', score[0])
print('Final model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 6s 62ms/step - loss: 2.7305 - accuracy: 0.1045 - val_loss: 2.3180 - val_accuracy: 0.1050
Epoch 2/100
72/72 [==============================] - 4s 58ms/step - loss: 2.5364 - accuracy: 0.1539 - val_loss: 2.3144 - val_accuracy: 0.1100
Epoch 3/100
72/72 [==============================] - 4s 58ms/step - loss: 2.3926 - accuracy: 0.1728 - val_loss: 2.2480 - val_accuracy: 0.1500
Epoch 4/100
72/72 [==============================] - 4s 59ms/step - loss: 2.2814 - accuracy: 0.2038 - val_loss: 2.1390 - val_accuracy: 0.2300
Epoch 5/100
72/72 [==============================] - 4s 58ms/step - loss: 2.1222 - accuracy: 0.2837 - val_loss: 2.0186 - val_accuracy: 0.3350
Epoch 6/100
72/72 [==============================] - 4s 58ms/step - loss: 1.9747 - accuracy: 0.3395 - val_loss: 1.9095 - val_accuracy: 0.3900
Epoch 7/100
72/72 [==============================] - 4s 59ms/step - loss: 1.8644 - accuracy: 0.3733 - val_loss: 1.8113 - val_accuracy: 0.4200
Epoch 